In [1]:
from __future__ import print_function

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [3]:
from ilqr.cost import QRCost, FiniteDiffCost
from ilqr.mujoco_dynamics import MujocoDynamics
from ilqr.mujoco_controller import iLQR, RecedingHorizonController
from ilqr.examples.cartpole import CartpoleDynamics
from ilqr.dynamics import constrain
from ilqr.utils.visualization import make_video_fn

from scipy.optimize import approx_fprime

import mujoco_py
from mujoco_py import MjViewer
import os

%load_ext autoreload
%autoreload 2

Choosing the latest nvidia driver: /usr/lib/nvidia-418, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-418']
Choosing the latest nvidia driver: /usr/lib/nvidia-418, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-418']


In [4]:
def on_iteration(iteration_count, xs, us, J_opt, accepted, converged):
    info = "converged" if converged else ("accepted" if accepted else "failed")
    final_state = xs[-1]
    print("iteration", iteration_count, info, J_opt)

In [5]:
xml_path = os.path.join('..', 'ilqr', 'xmls', 'half_cheetah.xml')
"""
State-Space (name/joint/parameter):
        0- rootx     slider      position (m)
        1- rootz     slider      position (m)
        2- rooty     hinge       angle (rad)
        3- bthigh    hinge       angle (rad)
        4- bshin     hinge       angle (rad)
        5- bfoot     hinge       angle (rad)
        6- fthigh    hinge       angle (rad)
        7- fshin     hinge       angle (rad)
        8- ffoot     hinge       angle (rad)
        9- rootx     slider      velocity (m/s)
        10- rootz     slider      velocity (m/s)
        11- rooty     hinge       angular velocity (rad/s)
        12- bthigh    hinge       angular velocity (rad/s)
        13- bshin     hinge       angular velocity (rad/s)
        14- bfoot     hinge       angular velocity (rad/s)
        15- fthigh    hinge       angular velocity (rad/s)
        16- fshin     hinge       angular velocity (rad/s)
        17- ffoot     hinge       angular velocity (rad/s)
"""

"""
Actuators (name/actuator/parameter):
        - bthigh    hinge       torque (N m)
        - bshin     hinge       torque (N m)
        - bfoot     hinge       torque (N m)
        - fthigh    hinge       torque (N m)
        - fshin     hinge       torque (N m)
        - ffoot     hinge       torque (N m)
"""

dynamics = MujocoDynamics(xml_path, frame_skip = 1, use_multiprocessing = True)
print('dt:', dynamics.dt)

Finished loading processFinished loading processFinished loading processFinished loading process  31642 
31641Finished loading process31645 31643

 31644
Finished loading process
Finished loading process  Finished loading process3167331681
 31674
Finished loading process
Finished loading process  dt: 0.01
Finished loading process 31656
Finished loading process31696
31688 
31689


In [6]:
def l(x, u, i):
    action_cost = np.square(u).sum()
    vel_cost = 10 * (x[9] - 4) ** 2
    steady_cost = 200 * (x[10] ** 2)
    return action_cost + vel_cost + steady_cost

cost2 = FiniteDiffCost(l, lambda x, i: l(x, np.array([0, 0, 0, 0, 0, 0]), i), 18, 6, use_multiprocessing = True)

Finished loading process Finished loading process31752
 Finished loading process31755
Finished loading process 31769 31762
Finished loading process
 31788
Finished loading processFinished loading process 31793Finished loading process 
31802 31807Finished loading process

Finished loading process 31822 Finished loading process 
31827Finished loading process31835 
31832



In [7]:
N = 200
x0 = dynamics.get_state()
us_init = np.random.uniform(-1, 1, (N, dynamics.action_size))
#us_init = np.zeros((N, dynamics.action_size))
ilqr = iLQR(dynamics, cost2, N)
mpc = RecedingHorizonController(x0, ilqr)

In [8]:
t0 = time.time()
J_hist = []
controls = mpc.control(us_init, step_size = 2, initial_n_iterations = 500, subsequent_n_iterations = 100, on_iteration = on_iteration)
us = []
for i in range(100):
    print('ITERATION', i, '\n')
    us.append(next(controls)[1])
print('time', time.time() - t0)
us = np.concatenate(us)

ITERATION 0 

iteration 0 failed 40403.829105577606
iteration 1 accepted 40376.96454270615
iteration 2 accepted 40345.06500840946
iteration 3 accepted 40320.8017700635
iteration 4 accepted 34223.5683006906
iteration 5 accepted 31978.313609422163
iteration 6 accepted 30231.43699723055
iteration 7 accepted 27618.980615100536
iteration 8 accepted 26243.5858798021
iteration 9 accepted 25382.77093405691
iteration 10 accepted 24259.044233949862
iteration 11 accepted 23536.76581558451
iteration 12 accepted 22165.988214877343
iteration 13 accepted 21362.845539746027
iteration 14 accepted 19878.850635084626
iteration 15 accepted 19811.37396792699
iteration 16 accepted 19536.022244410513
iteration 17 accepted 19270.180203558284
iteration 18 accepted 19013.421492500176
iteration 19 accepted 18772.012775871168
iteration 20 accepted 18533.4675259893
iteration 21 accepted 18293.477410730866
iteration 22 accepted 18069.249583035547
iteration 23 accepted 17792.6127240209
iteration 24 accepted 17592.07

iteration 203 accepted 8833.246209692215
iteration 204 accepted 8830.376913632492
iteration 205 accepted 8829.045585969016
iteration 206 accepted 8828.122804135208
iteration 207 accepted 8826.785633330905
iteration 208 accepted 8824.154795584565
iteration 209 failed 8824.154795584573
iteration 210 failed 8824.154795584573
iteration 211 failed 8824.154795584573
iteration 212 failed 8824.154795584573
iteration 213 failed 8824.154795584573
iteration 214 failed 8824.154795584573
iteration 215 failed 8824.154795584573
iteration 216 accepted 8823.967358443757
iteration 217 accepted 8823.565905051011
iteration 218 failed 8823.565905051008
iteration 219 failed 8823.565905051008
iteration 220 accepted 8822.887212236517
iteration 221 failed 8822.887212236517
iteration 222 failed 8822.887212236517
iteration 223 failed 8822.887212236517
iteration 224 accepted 8822.875917495106
iteration 225 failed 8822.87591749511
iteration 226 accepted 8822.746232018622
iteration 227 failed 8822.746232018624
iter

/home/wangjim19/ilqr/ilqr/mujoco_controller.py:167: UserWarning: exceeded max regularization term
  warnings.warn("exceeded max regularization term")


iteration 0 failed 8496.516752985572
iteration 1 failed 8496.516752985572
iteration 2 failed 8496.516752985572
iteration 3 failed 8496.516752985572
iteration 4 failed 8496.516752985572
iteration 5 failed 8496.516752985572
ITERATION 2 

iteration 0 failed 8137.865491379865
iteration 1 failed 8137.865491379865
iteration 2 failed 8137.865491379865
iteration 3 failed 8137.865491379865
iteration 4 failed 8137.865491379865
iteration 5 failed 8137.865491379865
ITERATION 3 

iteration 0 failed 7798.556044540344
iteration 1 failed 7798.556044540344
iteration 2 failed 7798.556044540344
iteration 3 failed 7798.556044540344
iteration 4 failed 7798.556044540344
iteration 5 failed 7798.556044540344
ITERATION 4 

iteration 0 failed 7474.553620150238
iteration 1 failed 7474.553620150238
iteration 2 failed 7474.553620150238
iteration 3 failed 7474.553620150238
iteration 4 accepted 7474.115318413826
iteration 5 accepted 7473.323397779773
iteration 6 failed 7473.323397779767
iteration 7 failed 7473.32339

iteration 4 accepted 5312.297809882948
iteration 5 accepted 5312.206404005016
iteration 6 accepted 5311.823794053085
iteration 7 failed 5311.823794053079
iteration 8 failed 5311.823794053079
iteration 9 failed 5311.823794053079
iteration 10 accepted 5310.28427393907
iteration 11 accepted 5309.439777601934
iteration 12 failed 5309.439777601932
iteration 13 failed 5309.439777601932
iteration 14 failed 5309.439777601932
iteration 15 accepted 5309.399139034034
iteration 16 accepted 5308.334513015478
iteration 17 failed 5308.334513015478
iteration 18 failed 5308.334513015478
iteration 19 accepted 5308.300795086186
iteration 20 accepted 5307.53058958526
iteration 21 failed 5307.530589585256
iteration 22 accepted 5304.852610376015
iteration 23 accepted 5302.443196743659
iteration 24 accepted 5299.390837517962
iteration 25 failed 5299.39083751796
iteration 26 accepted 5297.531051170547
iteration 27 failed 5297.531051170547
iteration 28 accepted 5296.532573571733
iteration 29 failed 5296.532573

...

iteration 21 failed 9088.808069187757
iteration 22 accepted 9087.898647274338
iteration 23 accepted 9003.789966311078
iteration 24 accepted 8980.206327149572
iteration 25 accepted 8949.258102259364
iteration 26 failed 8949.258102259364
iteration 27 failed 8949.258102259364
iteration 28 failed 8949.258102259364
iteration 29 failed 8949.258102259364
iteration 30 failed 8949.258102259364
iteration 31 accepted 8947.650076434737
iteration 32 failed 8947.650076434738
iteration 33 accepted 8946.502368472497
iteration 34 failed 8946.502368472497
iteration 35 accepted 8945.955210317112
iteration 36 failed 8945.955210317114
iteration 37 accepted 8945.311043296859
iteration 38 failed 8945.31104329686
iteration 39 accepted 8944.754871558507
iteration 40 failed 8944.754871558505
iteration 41 failed 8944.754871558505
iteration 42 accepted 8941.891639608459
iteration 43 accepted 8929.748197003451
iteration 44 failed 8929.748197003451
iteration 45 accepted 8929.698725097307
iteration 46 failed 8929.69

iteration 87 accepted 9001.175614772574
iteration 88 accepted 9000.89081051189
iteration 89 failed 9000.890810511894
iteration 90 accepted 9000.824732070356
iteration 91 failed 9000.824732070356
iteration 92 failed 9000.824732070356
iteration 93 accepted 9000.768368227747
iteration 94 accepted 9000.731864119925
iteration 95 failed 9000.731864119925
iteration 96 failed 9000.731864119925
iteration 97 failed 9000.731864119925
iteration 98 failed 9000.731864119925
iteration 99 accepted 8985.07832637112
ITERATION 89 

iteration 0 failed 9463.415065610392
iteration 1 failed 9463.415065610392
iteration 2 failed 9463.415065610392
iteration 3 accepted 9462.722413207888
iteration 4 accepted 9455.766792107988
iteration 5 accepted 9434.624923118576
iteration 6 accepted 9431.298909326952
iteration 7 accepted 9424.002260377845
iteration 8 failed 9424.002260377845
iteration 9 failed 9424.002260377845
iteration 10 failed 9424.002260377845
iteration 11 failed 9424.002260377845
iteration 12 accepted 941

iteration 48 failed 11947.287279609853
iteration 49 failed 11947.287279609853
iteration 50 accepted 11938.417443762986
iteration 51 accepted 11927.18481991427
iteration 52 failed 11927.184819914273
iteration 53 failed 11927.184819914273
iteration 54 accepted 11861.99084908279
iteration 55 failed 11861.990849082787
iteration 56 accepted 11855.095734305338
iteration 57 failed 11855.095734305338
iteration 58 accepted 11847.075876675885
iteration 59 failed 11847.075876675885
iteration 60 failed 11847.075876675885
iteration 61 accepted 11844.158195332997
iteration 62 failed 11844.158195333
iteration 63 failed 11844.158195333
iteration 64 failed 11844.158195333
iteration 65 accepted 11843.253450018616
iteration 66 failed 11843.253450018618
iteration 67 accepted 11842.35923933867
iteration 68 failed 11842.359239338668
iteration 69 accepted 11841.475978263612
iteration 70 failed 11841.47597826361
iteration 71 failed 11841.47597826361
iteration 72 accepted 11836.34000246013
iteration 73 failed 

iteration 81 accepted 11965.904117729875
iteration 82 accepted 11933.388293197338
iteration 83 failed 11933.388293197338
iteration 84 accepted 11927.70728341178
iteration 85 failed 11927.707283411775
iteration 86 failed 11927.707283411775
iteration 87 failed 11927.707283411775
iteration 88 accepted 11919.718648776472
iteration 89 accepted 11902.422698318456
iteration 90 accepted 11893.311061488721
iteration 91 failed 11893.311061488715
iteration 92 failed 11893.311061488715
iteration 93 failed 11893.311061488715
iteration 94 accepted 11879.227164269623
iteration 95 accepted 11866.432466404105
iteration 96 failed 11866.432466404109
iteration 97 failed 11866.432466404109
iteration 98 accepted 11838.059538866424
iteration 99 failed 11838.059538866424
ITERATION 96 

iteration 0 failed 12819.84933854008
iteration 1 failed 12819.84933854008
iteration 2 accepted 12819.43467753066
iteration 3 accepted 12814.382547462335
iteration 4 accepted 12803.994760748279
iteration 5 failed 12803.994760748

iteration 5 accepted 13610.075677804189
iteration 6 failed 13610.075677804194
iteration 7 accepted 13605.033891750596
iteration 8 accepted 13595.807682732362
iteration 9 failed 13595.807682732364
iteration 10 failed 13595.807682732364
iteration 11 accepted 12921.798064573315
iteration 12 accepted 12916.570249304696
iteration 13 failed 12916.570249304692
iteration 14 failed 12916.570249304692
iteration 15 accepted 12894.410521405094
iteration 16 failed 12894.410521405094
iteration 17 accepted 12871.513500178668
iteration 18 accepted 12864.879466979952
iteration 19 failed 12864.879466979954
iteration 20 accepted 12857.586567806025
iteration 21 accepted 12840.77338716639
iteration 22 failed 12840.773387166388
iteration 23 accepted 12774.168216519212
iteration 24 accepted 12728.827555565695
iteration 25 accepted 12696.286750838064
iteration 26 accepted 12665.107068862708
iteration 27 accepted 12659.155872665891
iteration 28 accepted 12632.657723492737
iteration 29 accepted 12627.0799917352

In [9]:
dynamics.sim.reset()
start = x0
start[0] += 3
dynamics.set_state(start)
print(dynamics.get_state())
video = []
for i in range(us.shape[0]):
    dynamics.step(us[i])
    video.append(dynamics.sim.render(512, 512))
    print(dynamics.get_state()[1], dynamics.get_state()[10], dynamics.get_state()[9])
    

[3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
-0.0024884530998890283 -0.24884530998890284 -0.04987122511350065
-0.005663888495031582 -0.3175435395142554 -0.08106375605465874
-0.009292725480238254 -0.36288369852066715 -0.11499473262836948
-0.01284599643588023 -0.3553270955641975 -0.01696948795659453
-0.016626395605568105 -0.37803991696878736 0.07851260742466151
-0.020447347593572546 -0.382095198800444 0.1221995309524199
-0.02400691387228762 -0.3559566278715072 0.1719572548002094
-0.026955483268245692 -0.2948569395958073 0.23011671525955357
-0.029944570341438083 -0.29890870731923913 0.2555582048551809
-0.03406542890852169 -0.41208585670836084 0.2557324579511906
-0.03298163915280983 0.10837897557118659 0.7328847244352675
-0.03533888519270215 -0.23572460398923262 0.5791825084554065
-0.039003198178352794 -0.36643129856506373 0.37343250982567366
-0.04300023897427527 -0.39970407959224746 0.20227911424227712
-0.04720347750678012 -0.4203238532504848 0.18193230539013205
-0.051712266525

-0.10217297867708768 -0.05112976467212999 4.040391246399635
-0.1027695579003082 -0.059657922322052664 3.9363723087335387
-0.10309809557159932 -0.032853767129112296 3.844450999362632
-0.1033584741506999 -0.026037857910057284 3.774940878352127
-0.1037150499275298 -0.03565757768298969 3.6797671735658617
-0.10407619116819783 -0.03611412406680252 3.657546043340447
-0.10383081983897134 0.024537132922649238 3.6421453246279683
-0.10343881612209681 0.039200371687453044 3.6367416964834653
-0.10312424019486977 0.03145759272270439 3.603655083407428
-0.10287248416933725 0.025175602553251727 3.5188266859920057
-0.10267382777867172 0.019865639066552795 3.5131882251735456
-0.10247397184590999 0.019985593276172695 3.5365920722812394
-0.10247542686234885 -0.0001455016438860965 3.6422986675503912
-0.10247696992677344 -0.00015430644245896275 3.7162722734226175
-0.10249120048609081 -0.0014230559317379608 3.768091064076475
-0.10253146860935893 -0.004026812326811898 3.7991955469994894
-0.10263610007772551 -0

In [10]:
make_video_fn(video)()

In [5]:
def run_mpc():
    xml_path = os.path.join('..', 'ilqr', 'xmls', 'half_cheetah.xml')
    """
    State-Space (name/joint/parameter):
            0- rootx     slider      position (m)
            1- rootz     slider      position (m)
            2- rooty     hinge       angle (rad)
            3- bthigh    hinge       angle (rad)
            4- bshin     hinge       angle (rad)
            5- bfoot     hinge       angle (rad)
            6- fthigh    hinge       angle (rad)
            7- fshin     hinge       angle (rad)
            8- ffoot     hinge       angle (rad)
            9- rootx     slider      velocity (m/s)
            10- rootz     slider      velocity (m/s)
            11- rooty     hinge       angular velocity (rad/s)
            12- bthigh    hinge       angular velocity (rad/s)
            13- bshin     hinge       angular velocity (rad/s)
            14- bfoot     hinge       angular velocity (rad/s)
            15- fthigh    hinge       angular velocity (rad/s)
            16- fshin     hinge       angular velocity (rad/s)
            17- ffoot     hinge       angular velocity (rad/s)
    """

    """
    Actuators (name/actuator/parameter):
            - bthigh    hinge       torque (N m)
            - bshin     hinge       torque (N m)
            - bfoot     hinge       torque (N m)
            - fthigh    hinge       torque (N m)
            - fshin     hinge       torque (N m)
            - ffoot     hinge       torque (N m)
    """

    dynamics = MujocoDynamics(xml_path, frame_skip = 1, use_multiprocessing = True)
    print('dt:', dynamics.dt)
    def l(x, u, i):
        action_cost = np.square(u).sum()
        vel_cost = 10 * (x[9] - 4) ** 2
        steady_cost = 200 * (x[10] ** 2)
        return action_cost + vel_cost + steady_cost

    cost2 = FiniteDiffCost(l, lambda x, i: l(x, np.array([0, 0, 0, 0, 0, 0]), i), 18, 6, use_multiprocessing = True)
    N = 200
    x0 = dynamics.get_state()
    us_init = np.random.uniform(-1, 1, (N, dynamics.action_size))
    #us_init = np.zeros((N, dynamics.action_size))
    ilqr = iLQR(dynamics, cost2, N)
    mpc = RecedingHorizonController(x0, ilqr)
    t0 = time.time()
    J_hist = []
    controls = mpc.control(us_init, step_size = 2, initial_n_iterations = 500, subsequent_n_iterations = 100, on_iteration = on_iteration)
    us = []
    for i in range(5):
        print('ITERATION', i, '\n')
        us.append(next(controls)[1])
    print('time', time.time() - t0)
    us = np.concatenate(us)
    
import cProfile
cProfile.run('run_mpc()')

Finished loading process 
24679Finished loading process Finished loading processFinished loading process 
24690Finished loading process24681  
24680
Finished loading process24683
 24682Finished loading process
 24693
Finished loading process 24710
Finished loading process dt: 0.01
24735
Finished loading processFinished loading process  2476624742

Finished loading processFinished loading processFinished loading process  2474924756 
24771
Finished loading process
 
Finished loading process24794 24809Finished loading process
 Finished loading process24818 
24827Finished loading process
 24834Finished loading process 
24845Finished loading process
 24854Finished loading process 
24863Finished loading process
 24872
Finished loading process 24881
ITERATION 0 

iteration 0 failed 40040.50480829799
iteration 1 failed 40040.50480829799
iteration 2 accepted 40030.52548619475
iteration 3 accepted 40009.38933315077
iteration 4 failed 40009.38933315077
iteration 5 failed 40009.38933315077
iterati

iteration 182 failed 11657.871695053502
iteration 183 failed 11657.871695053502


/home/wangjim19/ilqr/ilqr/mujoco_controller.py:167: UserWarning: exceeded max regularization term
  warnings.warn("exceeded max regularization term")


ITERATION 1 

iteration 0 accepted 11369.453871071848
iteration 1 accepted 11367.602535596054
iteration 2 accepted 11360.438259496557
iteration 3 accepted 11343.573173427098
iteration 4 accepted 11321.988230373718
iteration 5 accepted 11310.625352296807
iteration 6 accepted 11290.319251282683
iteration 7 accepted 11284.378375383369
iteration 8 accepted 11279.872924444691
iteration 9 accepted 11152.201149232791
iteration 10 accepted 11120.389662436424
iteration 11 accepted 11086.415069402468
iteration 12 accepted 10982.228011170462
iteration 13 accepted 10918.040394520476
iteration 14 accepted 10898.081419354301
iteration 15 accepted 10873.34030364976
iteration 16 accepted 10869.201935240346
iteration 17 accepted 10837.387300445436
iteration 18 accepted 10805.790446595382
iteration 19 accepted 10787.433644181008
iteration 20 accepted 10783.880595099987
iteration 21 accepted 10760.496966909284
iteration 22 accepted 10738.950440157982
iteration 23 accepted 10715.063012916684
iteration 24 

In [1]:
#HARD CODED COST DERIVATIVES

import multiprocessing as mp
import os
import numpy as np
from ilqr.cost import QRCost, FiniteDiffCost, Cost
from ilqr.mujoco_dynamics import MujocoDynamics
from ilqr.mujoco_controller import iLQR, RecedingHorizonController
from ilqr.examples.cartpole import CartpoleDynamics
from ilqr.dynamics import constrain

from scipy.optimize import approx_fprime

import mujoco_py
from mujoco_py import MjViewer
import time


class ExactCost(Cost):

    def __init__(self,
                 l,
                 l_terminal,
                 state_size,
                 action_size,
                 x_eps=None,
                 u_eps=None,
                 use_multiprocessing = False):
        
        self._l = l
        self._l_terminal = l_terminal
        self._state_size = state_size
        self._action_size = action_size

        self._x_eps = x_eps if x_eps else np.sqrt(np.finfo(float).eps)
        self._u_eps = u_eps if x_eps else np.sqrt(np.finfo(float).eps)

        self._x_eps_hess = np.sqrt(self._x_eps)
        self._u_eps_hess = np.sqrt(self._u_eps)

        self.multiprocessing = use_multiprocessing
        if self.multiprocessing:
            self._pool = mp.Pool(initializer = ExactCost._worker_init,
                                 initargs = (l, l_terminal, state_size, action_size, x_eps, u_eps, False))

        super(ExactCost, self).__init__()

    @staticmethod
    def _worker_init(l,
                     l_terminal,
                     state_size,
                     action_size,
                     x_eps,
                     u_eps,
                     use_multiprocessing):
        """
        Initializes sims for workers in multiprocessing Pool.
        """
        global cost
        cost = ExactCost(l, l_terminal, state_size, action_size, x_eps, u_eps, use_multiprocessing)
        print("Finished loading process", os.getpid())

    @staticmethod
    def _worker(x, u, i):
        return (cost.l(x, u, i), cost.l_x(x, u, i), cost.l_u(x, u, i), cost.l_xx(x, u, i), cost.l_ux(x, u, i), cost.l_uu(x, u, i))

    def l_derivs(self, xs, us):
        if self.multiprocessing:
            results = self._pool.starmap(ExactCost._worker, [(xs[i], us[i], i) for i in range(us.shape[0])], chunksize = us.shape[0] // mp.cpu_count())
            return ([result[0] for result in results],
                    [result[1] for result in results],
                    [result[2] for result in results],
                    [result[3] for result in results],
                    [result[4] for result in results],
                    [result[5] for result in results])

        L = [self.l(xs[i], us[i], i) for i in range(us.shape[0])]
        L_x = [self.l_x(xs[i], us[i], i) for i in range(us.shape[0])]
        L_u = [self.l_u(xs[i], us[i], i) for i in range(us.shape[0])]
        L_xx = [self.l_xx(xs[i], us[i], i) for i in range(us.shape[0])]
        L_ux = [self.l_ux(xs[i], us[i], i) for i in range(us.shape[0])]
        L_uu = [self.l_uu(xs[i], us[i], i) for i in range(us.shape[0])]
        return (L, L_x, L_u, L_xx, L_ux, L_uu)

    def l(self, x, u, i, terminal=False):
        """Instantaneous cost function.

        Args:
            x: Current state [state_size].
            u: Current control [action_size]. None if terminal.
            i: Current time step.
            terminal: Compute terminal cost. Default: False.

        Returns:
            Instantaneous cost (scalar).
        """
        if terminal:
            return self._l_terminal(x, i)

        return self._l(x, u, i)

    
    def l_x(self, x, u, i, terminal=False):
        deriv = np.zeros(18)
        deriv[9] = 20 * (x[9] - 4)
        deriv[10] = 400 * x[10]
        return deriv
    def l_u(self, x, u, i, terminal = False):
        if terminal:
            return np.zeros(6)
        return 2 * u
    def l_xx(self, x, u, i, terminal=False):
        deriv = np.zeros((18, 18))
        deriv[9][9] = 20
        deriv[10][10] = 400
        return deriv
    def l_ux(self, x, u, i, terminal=False):
        return np.zeros((6, 18))
    def l_uu(self, x, u, i, terminal=False):
        if terminal:
            return np.zeros((6, 6))
        return 2 * np.eye(6)

def on_iteration(iteration_count, xs, us, J_opt, accepted, converged):
    info = "converged" if converged else ("accepted" if accepted else "failed")
    final_state = xs[-1]
    print("iteration", iteration_count, info, J_opt)
    
    
    
def run_mpc():
    xml_path = os.path.join('..', 'ilqr', 'xmls', 'half_cheetah.xml')
    """
    State-Space (name/joint/parameter):
            0- rootx     slider      position (m)
            1- rootz     slider      position (m)
            2- rooty     hinge       angle (rad)
            3- bthigh    hinge       angle (rad)
            4- bshin     hinge       angle (rad)
            5- bfoot     hinge       angle (rad)
            6- fthigh    hinge       angle (rad)
            7- fshin     hinge       angle (rad)
            8- ffoot     hinge       angle (rad)
            9- rootx     slider      velocity (m/s)
            10- rootz     slider      velocity (m/s)
            11- rooty     hinge       angular velocity (rad/s)
            12- bthigh    hinge       angular velocity (rad/s)
            13- bshin     hinge       angular velocity (rad/s)
            14- bfoot     hinge       angular velocity (rad/s)
            15- fthigh    hinge       angular velocity (rad/s)
            16- fshin     hinge       angular velocity (rad/s)
            17- ffoot     hinge       angular velocity (rad/s)
    """

    """
    Actuators (name/actuator/parameter):
            - bthigh    hinge       torque (N m)
            - bshin     hinge       torque (N m)
            - bfoot     hinge       torque (N m)
            - fthigh    hinge       torque (N m)
            - fshin     hinge       torque (N m)
            - ffoot     hinge       torque (N m)
    """

    dynamics = MujocoDynamics(xml_path, frame_skip = 1, use_multiprocessing = True)
    print('dt:', dynamics.dt)
    def l(x, u, i):
        action_cost = np.square(u).sum()
        vel_cost = 10 * (x[9] - 4) ** 2
        steady_cost = 200 * (x[10] ** 2)
        return action_cost + vel_cost + steady_cost

    cost = ExactCost(l, lambda x, i: l(x, np.array([0, 0, 0, 0, 0, 0]), i), 18, 6, use_multiprocessing = True)
    N = 200
    x0 = dynamics.get_state()
    us_init = np.random.uniform(-1, 1, (N, dynamics.action_size))
    #us_init = np.zeros((N, dynamics.action_size))
    ilqr = iLQR(dynamics, cost, N)
    mpc = RecedingHorizonController(x0, ilqr)
    t0 = time.time()
    J_hist = []
    controls = mpc.control(us_init, step_size = 2, initial_n_iterations = 500, subsequent_n_iterations = 100, on_iteration = on_iteration)
    us = []
    for i in range(5):
        print('ITERATION', i, '\n')
        us.append(next(controls)[1])
    print('time', time.time() - t0)
    us = np.concatenate(us)
    
import cProfile
cProfile.run('run_mpc()')

Choosing the latest nvidia driver: /usr/lib/nvidia-418, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-418']
Choosing the latest nvidia driver: /usr/lib/nvidia-418, among ['/usr/lib/nvidia-375', '/usr/lib/nvidia-418']
dt: 0.01
Finished loading processFinished loading process Finished loading process 24369Finished loading process24386
Finished loading process Finished loading process  Finished loading process
24373 Finished loading processFinished loading process2440024370Finished loading process  24393

 
Finished loading process244112437524426 
 Finished loading process

24374
Finished loading process  24371

24465
Finished loading process24372
 24377Finished loading processFinished loading process
  2449224380Finished loading process

Finished loading process 24505 
Finished loading process24518
Finished loading processFinished loading processFinished loading process    243782452724379Finished loading process24376


 
24546
Finished loading process 24569
ITERATION 0 

iteration 0 acc

iteration 179 accepted 8696.246067281143
iteration 180 accepted 8694.693636031012
iteration 181 accepted 8693.543346538341
iteration 182 accepted 8693.287186897527
iteration 183 accepted 8690.867647576539
iteration 184 accepted 8690.451055549123
iteration 185 accepted 8688.829425711627
iteration 186 accepted 8687.58653895497
iteration 187 accepted 8686.636353734015
iteration 188 accepted 8684.7354504144
iteration 189 accepted 8683.349611850426
iteration 190 accepted 8682.17164093477
iteration 191 accepted 8681.442404609033
iteration 192 accepted 8680.254333420822
iteration 193 accepted 8678.829953819059
iteration 194 accepted 8677.140709328185
iteration 195 accepted 8676.985375890115
iteration 196 accepted 8674.810823489814
iteration 197 accepted 8674.456768034037
iteration 198 accepted 8672.445130240309
iteration 199 accepted 8671.156760633012
iteration 200 accepted 8670.417821180772
iteration 201 accepted 8669.52366150155
iteration 202 accepted 8667.805933435904
iteration 203 accepte

/home/wangjim19/ilqr/ilqr/mujoco_controller.py:167: UserWarning: exceeded max regularization term
  warnings.warn("exceeded max regularization term")


ITERATION 1 

iteration 0 failed 8331.747727226411
iteration 1 failed 8331.747727226411
iteration 2 failed 8331.747727226411
iteration 3 failed 8331.747727226411
iteration 4 accepted 8331.649299155915
iteration 5 accepted 8331.296739910384
iteration 6 accepted 8330.841315499767
iteration 7 failed 8330.841315499765
iteration 8 accepted 8329.232110306548
iteration 9 failed 8329.232110306542
iteration 10 accepted 8328.981217580518
iteration 11 failed 8328.98121758053
iteration 12 failed 8328.98121758053
iteration 13 accepted 8328.26233462382
iteration 14 accepted 8327.727915667097
iteration 15 failed 8327.727915667096
iteration 16 failed 8327.727915667096
iteration 17 accepted 8326.567075550623
iteration 18 accepted 8326.526953766455
iteration 19 accepted 8325.292190646078
iteration 20 accepted 8323.391010991068
iteration 21 failed 8323.391010991068
iteration 22 failed 8323.391010991068
iteration 23 failed 8323.391010991068
iteration 24 accepted 8321.17660001454
iteration 25 failed 8321.1

      576    0.005    0.000    0.027    0.000 pool.py:368(_map_async)
        1    0.000    0.000    0.000    0.000 pool.py:53(RemoteTraceback)
        1    0.000    0.000    0.000    0.000 pool.py:59(ExceptionWithTraceback)
        1    0.000    0.000    0.000    0.000 pool.py:617(ApplyResult)
      576    0.002    0.000    0.009    0.000 pool.py:619(__init__)
      576    0.001    0.000    0.001    0.000 pool.py:627(ready)
      576    0.002    0.000   11.904    0.021 pool.py:634(wait)
      576    0.001    0.000   11.907    0.021 pool.py:637(get)
        1    0.000    0.000    0.000    0.000 pool.py:661(MapResult)
      576    0.004    0.000    0.013    0.000 pool.py:663(__init__)
        1    0.000    0.000    0.000    0.000 pool.py:702(IMapIterator)
        1    0.000    0.000    0.000    0.000 pool.py:76(MaybeEncodingError)
        1    0.000    0.000    0.000    0.000 pool.py:766(IMapUnorderedIterator)
        1    0.000    0.000    0.000    0.000 pool.py:780(ThreadPool)
       

In [1]:
print(10 * (5-4) ** 2)

10
